# Todo (Issue)
- answer code 누적합 column 제작
- 그래프 데이터 만들기
- 해당 시험을 몇 번째 푸는지 여부 (11/21 완)
   - 해당 시험을 몇 번째 푸는지 여부 (11/21 완)
   - 시험 -> 문제, 태그로 변환도 가능하며 이전에 맞췄는지 여부도 포함 (11/21 완)
- 문제를 푸는데 걸린시간이 평균 시간보다 오래걸렸는지 여부 추가
   - 한 유저가 어떤 문제를 풀때 그 문제를 푸는데 걸린 시간이 그 문제의 평균 시간보다 적게 걸렸는지 혹은 오래 걸렸는지 여부를 column에 추가
   - 해당 문항을 맞춘 학생의 평균 시간과 틀린 학생의 평균 시간 2가지를 Feature로 주어 활용하는 방안도 있음
- 사용자의 정답률 column 제작 (11/21 완)
   - 누적 정답률, 최근 정답률 두 가지를 제작한다. 최근 정답률은 window size를 줘서 최근 N개의 문항에 대한 평균을 구할 수 있도록 (11/21 완, N개 문항은 모델 부분에서 적용)
- 문항, 시험지, 태그의 평균 정답률 (11/21 완)
   - [EDA] test에서 정답을 예측해야 되는 문제가 train에도 있는지 확인
- 문제를 푼 사용자와 사용자가 푼 문항을 MF나 SVD를 통해 user-item matrix 생성해 feature로 사용

# Todo (추가)
- 장원준 EDA (11/21 완)
   - 각 문항 번호 column 추가
   - 각 유저별 푼 카테고리 목록 추가
- 정준환 EDA
   - timestamp이용해 걸린 시간 계산
   - 오류 시간은 -1 처리, threshold 걸기(200초)
- 이수경 EDA
   - timestamp이용해 걸린 시간 계산(elapsed_time)
   - 한 유저가 동일한 시험지를 두 번 이상 풀었을 경우를 고려 못 해줌 => 고려 필요
   - 한 유저가 동일한 시험지를 연속해서 풀었을 수도 있음 => 확인 필요 => 288번 있음
   - 문제 번호는 랜덤이라는 것 주의해서 처리
   - 시간 이상치 처리(threshold 200초, 평균 처리 등)
   - 7863번 knowledgetag 제외 모두 고유한 category를 가진다는 것 유념
- 류명현 EDA
   - 시간대별 정답률 다르므로 timestamp를 나누자



In [72]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from tqdm import tqdm

In [73]:
train = pd.read_csv('data/train_data.csv')
test = pd.read_csv('data/test_data.csv')
submission = pd.read_csv('data/sample_submission.csv')

In [74]:
train
new_train = train.copy()
new_train

,userID,assessmentItemID,testId,answerCode,Timestamp,KnowledgeTag
0,0,A060001001,A060000001,1,2020-03-24 00:17:11,7224
1,0,A060001002,A060000001,1,2020-03-24 00:17:14,7225
2,0,A060001003,A060000001,1,2020-03-24 00:17:22,7225
3,0,A060001004,A060000001,1,2020-03-24 00:17:29,7225
4,0,A060001005,A060000001,1,2020-03-24 00:17:36,7225
...,...,...,...,...,...,...
2266581,7441,A030071005,A030000071,0,2020-06-05 06:50:21,438
2266582,7441,A040165001,A040000165,1,2020-08-21 01:06:39,8836
2266583,7441,A040165002,A040000165,1,2020-08-21 01:06:50,8836
2266584,7441,A040165003,A040000165,1,2020-08-21 01:07:36,8836


# 기본 전처리
- 새로운 column 추가
   - question_number: 문제가 테스트 상에서 몇 번인지
   - test_cat: 시험의 대분류 category가 몇인지
   - test_id: 시험지 고유의 번호
   - test_month: Timestamp에 찍힌 달
   - test_day: Timestamp에 찍힌 일
   - test_hour: Timestamp에 찍힌 시간
- 날짜 데이터 변경
   - 2019년 12월 31일 데이터에 대해 2020년 1월 1일로 변경 후 test_year는 버림

In [75]:
new_train['question_number'] = new_train['assessmentItemID'].apply(lambda x: x[8:10]).map(int) # 문제 순서 번호
new_train['test_cat'] = new_train['testId'].apply(lambda x: x[2]) # 시험지 대분류
new_train['test_id'] = new_train['assessmentItemID'].apply(lambda x: x[4:7]) # 시험지 고유 번호
new_train["Timestamp"] = new_train["Timestamp"].astype("str") # timestamp 처리

In [76]:
new_train['test_year'] = new_train['Timestamp'].apply(lambda x: x[:4])
new_train['test_day'] = new_train['Timestamp'].apply(lambda x: x[8:10])
new_train['test_hour'] = new_train['Timestamp'].apply(lambda x: x[11:13])
new_train['test_month'] = new_train['Timestamp'].apply(lambda x: x[5:7])

In [77]:
# timestamp에서 2019년 데이터는 없앰
new_train.loc[new_train['Timestamp'].str.contains('2019'), 'test_year'] = '2020'
new_train.loc[new_train['Timestamp'].str.contains('2019'), 'test_month'] = '01'
new_train.loc[new_train['Timestamp'].str.contains('2019'), 'test_day'] = '01'

new_train[new_train['Timestamp'].str.contains('2019')]

,userID,assessmentItemID,testId,answerCode,Timestamp,KnowledgeTag,question_number,test_cat,test_id,test_year,test_day,test_hour,test_month
69406,92,A020036001,A020000036,1,2019-12-31 23:17:24,7696,1,2,036,2020,01,23,01
69407,92,A020036002,A020000036,1,2019-12-31 23:17:50,7696,2,2,036,2020,01,23,01
69408,92,A020036003,A020000036,1,2019-12-31 23:18:12,7696,3,2,036,2020,01,23,01
69409,92,A020036004,A020000036,1,2019-12-31 23:18:53,7696,4,2,036,2020,01,23,01
69410,92,A020036005,A020000036,1,2019-12-31 23:20:40,7696,5,2,036,2020,01,23,01
69411,92,A020036007,A020000036,1,2019-12-31 23:22:15,7696,7,2,036,2020,01,23,01
69412,92,A020036008,A020000036,1,2019-12-31 23:23:43,7696,8,2,036,2020,01,23,01
69413,92,A020036009,A020000036,0,2019-12-31 23:24:41,7696,9,2,036,2020,01,23,01
555508,784,A060014001,A060000014,1,2019-12-31 15:08:01,7229,1,6,014,2020,01,15,01
555509,784,A060014002,A060000014,1,2019-12-31 15:08:21,7229,2,6,014,2020,01,15,01


In [78]:
new_train = new_train.drop(['test_year'], axis = 1)
new_train

,userID,assessmentItemID,testId,answerCode,Timestamp,KnowledgeTag,question_number,test_cat,test_id,test_day,test_hour,test_month
0,0,A060001001,A060000001,1,2020-03-24 00:17:11,7224,1,6,001,24,00,03
1,0,A060001002,A060000001,1,2020-03-24 00:17:14,7225,2,6,001,24,00,03
2,0,A060001003,A060000001,1,2020-03-24 00:17:22,7225,3,6,001,24,00,03
3,0,A060001004,A060000001,1,2020-03-24 00:17:29,7225,4,6,001,24,00,03
4,0,A060001005,A060000001,1,2020-03-24 00:17:36,7225,5,6,001,24,00,03
...,...,...,...,...,...,...,...,...,...,...,...,...
2266581,7441,A030071005,A030000071,0,2020-06-05 06:50:21,438,5,3,071,05,06,06
2266582,7441,A040165001,A040000165,1,2020-08-21 01:06:39,8836,1,4,165,21,01,08
2266583,7441,A040165002,A040000165,1,2020-08-21 01:06:50,8836,2,4,165,21,01,08
2266584,7441,A040165003,A040000165,1,2020-08-21 01:07:36,8836,3,4,165,21,01,08


# [전처리] 문항, 시험지, 태그의 평균 정답률#14

- new_train2 생성

In [79]:
# 문항 시험지 태그의 평균정답률
# 시험지의 평균 정답률
test_acc = new_train.groupby(['test_cat', 'test_id'])['answerCode'].mean().reset_index()
test_acc.columns = ['test_cat','test_id','test_acc']
new_train2 = pd.merge(left = new_train, right = test_acc, on = ['test_cat', 'test_id'], how = 'left')

# 문항의 평균 정답률
question_acc = new_train.groupby(['test_cat', 'test_id', 'question_number'])['answerCode'].mean().reset_index()
question_acc.columns = ['test_cat', 'test_id', 'question_number', 'question_acc']
new_train2 = pd.merge(left = new_train2, right = question_acc, on = ['test_cat', 'test_id', 'question_number'], how = 'left')

# 태그의 평균 정답률
tag_acc = new_train.groupby(['KnowledgeTag'])['answerCode'].mean().reset_index()
tag_acc.columns = ['KnowledgeTag', 'tag_acc']
new_train2 = pd.merge(left = new_train2, right = tag_acc, on = ['KnowledgeTag'], how = 'left')

In [80]:
new_train2

,userID,assessmentItemID,testId,answerCode,Timestamp,KnowledgeTag,question_number,test_cat,test_id,test_day,test_hour,test_month,test_acc,question_acc,tag_acc
0,0,A060001001,A060000001,1,2020-03-24 00:17:11,7224,1,6,001,24,00,03,0.947683,0.982063,0.955022
1,0,A060001002,A060000001,1,2020-03-24 00:17:14,7225,2,6,001,24,00,03,0.947683,0.964126,0.913187
2,0,A060001003,A060000001,1,2020-03-24 00:17:22,7225,3,6,001,24,00,03,0.947683,0.910314,0.913187
3,0,A060001004,A060000001,1,2020-03-24 00:17:29,7225,4,6,001,24,00,03,0.947683,0.968610,0.913187
4,0,A060001005,A060000001,1,2020-03-24 00:17:36,7225,5,6,001,24,00,03,0.947683,0.941704,0.913187
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2266581,7441,A030071005,A030000071,0,2020-06-05 06:50:21,438,5,3,071,05,06,06,0.662590,0.435252,0.689706
2266582,7441,A040165001,A040000165,1,2020-08-21 01:06:39,8836,1,4,165,21,01,08,0.655109,0.649635,0.697874
2266583,7441,A040165002,A040000165,1,2020-08-21 01:06:50,8836,2,4,165,21,01,08,0.655109,0.635036,0.697874
2266584,7441,A040165003,A040000165,1,2020-08-21 01:07:36,8836,3,4,165,21,01,08,0.655109,0.791971,0.697874


In [81]:
# 시간별 정답률
time_acc = new_train.groupby(['test_hour'])['answerCode'].mean().reset_index()
time_acc.columns = ['test_hour', 'time_acc']
new_train2 = pd.merge(left = new_train2, right = time_acc, on = ['test_hour'], how = 'left')

In [82]:
# 월별 정답률
month_acc = new_train.groupby(['test_month'])['answerCode'].mean().reset_index()
month_acc.columns = ['test_month', 'month_acc']
new_train2 = pd.merge(left = new_train2, right = month_acc, on = ['test_month'], how = 'left')

In [83]:
# 누적 정답률
user_acc = new_train.groupby('userID')['answerCode'].mean().reset_index()
user_acc.columns = ['userID', 'user_acc']
new_train2 = pd.merge(left = new_train2, right = user_acc, on = 'userID', how = 'left')

In [84]:
new_train2

,userID,assessmentItemID,testId,answerCode,Timestamp,KnowledgeTag,question_number,test_cat,test_id,test_day,test_hour,test_month,test_acc,question_acc,tag_acc,time_acc,month_acc,user_acc
0,0,A060001001,A060000001,1,2020-03-24 00:17:11,7224,1,6,001,24,00,03,0.947683,0.982063,0.955022,0.649446,0.682643,0.630872
1,0,A060001002,A060000001,1,2020-03-24 00:17:14,7225,2,6,001,24,00,03,0.947683,0.964126,0.913187,0.649446,0.682643,0.630872
2,0,A060001003,A060000001,1,2020-03-24 00:17:22,7225,3,6,001,24,00,03,0.947683,0.910314,0.913187,0.649446,0.682643,0.630872
3,0,A060001004,A060000001,1,2020-03-24 00:17:29,7225,4,6,001,24,00,03,0.947683,0.968610,0.913187,0.649446,0.682643,0.630872
4,0,A060001005,A060000001,1,2020-03-24 00:17:36,7225,5,6,001,24,00,03,0.947683,0.941704,0.913187,0.649446,0.682643,0.630872
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2266581,7441,A030071005,A030000071,0,2020-06-05 06:50:21,438,5,3,071,05,06,06,0.662590,0.435252,0.689706,0.629941,0.651967,0.555556
2266582,7441,A040165001,A040000165,1,2020-08-21 01:06:39,8836,1,4,165,21,01,08,0.655109,0.649635,0.697874,0.644321,0.657599,0.555556
2266583,7441,A040165002,A040000165,1,2020-08-21 01:06:50,8836,2,4,165,21,01,08,0.655109,0.635036,0.697874,0.644321,0.657599,0.555556
2266584,7441,A040165003,A040000165,1,2020-08-21 01:07:36,8836,3,4,165,21,01,08,0.655109,0.791971,0.697874,0.644321,0.657599,0.555556


In [105]:
new_train3 = new_train2.copy()

# [전처리] 해당 시험/문제/태그를 몇 번째 푸는지 여부

In [106]:
# 문제를 몇 번 푸는지?
exp_question = new_train3.groupby(['test_cat', 'test_id', 'question_number'])['userID'].value_counts().reset_index(name='exp_question')
new_train3 = pd.merge(left = new_train3, right = exp_question, on = ['userID', 'test_cat', 'test_id', 'question_number'], how = 'left')

In [107]:
new_train3.head(10)

,userID,assessmentItemID,testId,answerCode,Timestamp,KnowledgeTag,question_number,test_cat,test_id,test_day,test_hour,test_month,test_acc,question_acc,tag_acc,time_acc,month_acc,user_acc,exp_question
0,0,A060001001,A060000001,1,2020-03-24 00:17:11,7224,1,6,001,24,00,03,0.947683,0.982063,0.955022,0.649446,0.682643,0.630872,1
1,0,A060001002,A060000001,1,2020-03-24 00:17:14,7225,2,6,001,24,00,03,0.947683,0.964126,0.913187,0.649446,0.682643,0.630872,1
2,0,A060001003,A060000001,1,2020-03-24 00:17:22,7225,3,6,001,24,00,03,0.947683,0.910314,0.913187,0.649446,0.682643,0.630872,1
3,0,A060001004,A060000001,1,2020-03-24 00:17:29,7225,4,6,001,24,00,03,0.947683,0.968610,0.913187,0.649446,0.682643,0.630872,1
4,0,A060001005,A060000001,1,2020-03-24 00:17:36,7225,5,6,001,24,00,03,0.947683,0.941704,0.913187,0.649446,0.682643,0.630872,1
5,0,A060001007,A060000001,1,2020-03-24 00:17:47,7225,7,6,001,24,00,03,0.947683,0.919283,0.913187,0.649446,0.682643,0.630872,1
6,0,A060003001,A060000003,0,2020-03-26 05:52:03,7226,1,6,003,26,05,03,0.790562,0.882353,0.799552,0.624776,0.682643,0.630872,1
7,0,A060003002,A060000003,1,2020-03-26 05:52:10,7226,2,6,003,26,05,03,0.790562,0.918552,0.799552,0.624776,0.682643,0.630872,1
8,0,A060003003,A060000003,1,2020-03-26 05:53:14,7226,3,6,003,26,05,03,0.790562,0.520362,0.799552,0.624776,0.682643,0.630872,1
9,0,A060003004,A060000003,1,2020-03-26 05:53:29,7226,4,6,003,26,05,03,0.790562,0.823529,0.799552,0.624776,0.682643,0.630872,1


In [108]:

# 시험을 몇 번 푸는지?

fff = new_train3.groupby(['test_cat', 'test_id'])['question_number'].unique().reset_index()
fff['len'] = fff['question_number'].apply(len)

# new_train3['len'] = new_train3['question_number'].apply(len)
# test2len = { key:value for key, value in zip(new_train3['testId'], new_train3['len'])}
# train['question_len'] = train['testId'].map(test2len)

exp_test = new_train3.groupby(['test_cat', 'test_id'])['userID'].value_counts().reset_index(name='exp_test')
new_train3 = pd.merge(left = new_train3, right = exp_test, on = ['userID', 'test_cat', 'test_id'], how = 'left')
new_train3 = pd.merge(left = new_train3, right = fff, on = ['test_cat', 'test_id'], how = 'left')
new_train3


,userID,assessmentItemID,testId,answerCode,Timestamp,KnowledgeTag,question_number_x,test_cat,test_id,test_day,...,test_acc,question_acc,tag_acc,time_acc,month_acc,user_acc,exp_question,exp_test,question_number_y,len
0,0,A060001001,A060000001,1,2020-03-24 00:17:11,7224,1,6,001,24,...,0.947683,0.982063,0.955022,0.649446,0.682643,0.630872,1,6,"[1, 2, 3, 4, 5, 7]",6
1,0,A060001002,A060000001,1,2020-03-24 00:17:14,7225,2,6,001,24,...,0.947683,0.964126,0.913187,0.649446,0.682643,0.630872,1,6,"[1, 2, 3, 4, 5, 7]",6
2,0,A060001003,A060000001,1,2020-03-24 00:17:22,7225,3,6,001,24,...,0.947683,0.910314,0.913187,0.649446,0.682643,0.630872,1,6,"[1, 2, 3, 4, 5, 7]",6
3,0,A060001004,A060000001,1,2020-03-24 00:17:29,7225,4,6,001,24,...,0.947683,0.968610,0.913187,0.649446,0.682643,0.630872,1,6,"[1, 2, 3, 4, 5, 7]",6
4,0,A060001005,A060000001,1,2020-03-24 00:17:36,7225,5,6,001,24,...,0.947683,0.941704,0.913187,0.649446,0.682643,0.630872,1,6,"[1, 2, 3, 4, 5, 7]",6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2266581,7441,A030071005,A030000071,0,2020-06-05 06:50:21,438,5,3,071,05,...,0.662590,0.435252,0.689706,0.629941,0.651967,0.555556,1,5,"[1, 2, 3, 4, 5]",5
2266582,7441,A040165001,A040000165,1,2020-08-21 01:06:39,8836,1,4,165,21,...,0.655109,0.649635,0.697874,0.644321,0.657599,0.555556,1,4,"[1, 2, 3, 4]",4
2266583,7441,A040165002,A040000165,1,2020-08-21 01:06:50,8836,2,4,165,21,...,0.655109,0.635036,0.697874,0.644321,0.657599,0.555556,1,4,"[1, 2, 3, 4]",4
2266584,7441,A040165003,A040000165,1,2020-08-21 01:07:36,8836,3,4,165,21,...,0.655109,0.791971,0.697874,0.644321,0.657599,0.555556,1,4,"[1, 2, 3, 4]",4


In [109]:
new_train3

,userID,assessmentItemID,testId,answerCode,Timestamp,KnowledgeTag,question_number_x,test_cat,test_id,test_day,...,test_acc,question_acc,tag_acc,time_acc,month_acc,user_acc,exp_question,exp_test,question_number_y,len
0,0,A060001001,A060000001,1,2020-03-24 00:17:11,7224,1,6,001,24,...,0.947683,0.982063,0.955022,0.649446,0.682643,0.630872,1,6,"[1, 2, 3, 4, 5, 7]",6
1,0,A060001002,A060000001,1,2020-03-24 00:17:14,7225,2,6,001,24,...,0.947683,0.964126,0.913187,0.649446,0.682643,0.630872,1,6,"[1, 2, 3, 4, 5, 7]",6
2,0,A060001003,A060000001,1,2020-03-24 00:17:22,7225,3,6,001,24,...,0.947683,0.910314,0.913187,0.649446,0.682643,0.630872,1,6,"[1, 2, 3, 4, 5, 7]",6
3,0,A060001004,A060000001,1,2020-03-24 00:17:29,7225,4,6,001,24,...,0.947683,0.968610,0.913187,0.649446,0.682643,0.630872,1,6,"[1, 2, 3, 4, 5, 7]",6
4,0,A060001005,A060000001,1,2020-03-24 00:17:36,7225,5,6,001,24,...,0.947683,0.941704,0.913187,0.649446,0.682643,0.630872,1,6,"[1, 2, 3, 4, 5, 7]",6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2266581,7441,A030071005,A030000071,0,2020-06-05 06:50:21,438,5,3,071,05,...,0.662590,0.435252,0.689706,0.629941,0.651967,0.555556,1,5,"[1, 2, 3, 4, 5]",5
2266582,7441,A040165001,A040000165,1,2020-08-21 01:06:39,8836,1,4,165,21,...,0.655109,0.649635,0.697874,0.644321,0.657599,0.555556,1,4,"[1, 2, 3, 4]",4
2266583,7441,A040165002,A040000165,1,2020-08-21 01:06:50,8836,2,4,165,21,...,0.655109,0.635036,0.697874,0.644321,0.657599,0.555556,1,4,"[1, 2, 3, 4]",4
2266584,7441,A040165003,A040000165,1,2020-08-21 01:07:36,8836,3,4,165,21,...,0.655109,0.791971,0.697874,0.644321,0.657599,0.555556,1,4,"[1, 2, 3, 4]",4


In [110]:
type(new_train3['len'][0])

numpy.int64

In [111]:
new_train3['exp_test'] = new_train3.apply(lambda x: x['exp_test'] / x['len'], axis = 1)
# df['column3'] = df.apply(lambda x: cos_sim(x['column1'], x['column2']), axis=1)
new_train3

,userID,assessmentItemID,testId,answerCode,Timestamp,KnowledgeTag,question_number_x,test_cat,test_id,test_day,...,test_acc,question_acc,tag_acc,time_acc,month_acc,user_acc,exp_question,exp_test,question_number_y,len
0,0,A060001001,A060000001,1,2020-03-24 00:17:11,7224,1,6,001,24,...,0.947683,0.982063,0.955022,0.649446,0.682643,0.630872,1,1.0,"[1, 2, 3, 4, 5, 7]",6
1,0,A060001002,A060000001,1,2020-03-24 00:17:14,7225,2,6,001,24,...,0.947683,0.964126,0.913187,0.649446,0.682643,0.630872,1,1.0,"[1, 2, 3, 4, 5, 7]",6
2,0,A060001003,A060000001,1,2020-03-24 00:17:22,7225,3,6,001,24,...,0.947683,0.910314,0.913187,0.649446,0.682643,0.630872,1,1.0,"[1, 2, 3, 4, 5, 7]",6
3,0,A060001004,A060000001,1,2020-03-24 00:17:29,7225,4,6,001,24,...,0.947683,0.968610,0.913187,0.649446,0.682643,0.630872,1,1.0,"[1, 2, 3, 4, 5, 7]",6
4,0,A060001005,A060000001,1,2020-03-24 00:17:36,7225,5,6,001,24,...,0.947683,0.941704,0.913187,0.649446,0.682643,0.630872,1,1.0,"[1, 2, 3, 4, 5, 7]",6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2266581,7441,A030071005,A030000071,0,2020-06-05 06:50:21,438,5,3,071,05,...,0.662590,0.435252,0.689706,0.629941,0.651967,0.555556,1,1.0,"[1, 2, 3, 4, 5]",5
2266582,7441,A040165001,A040000165,1,2020-08-21 01:06:39,8836,1,4,165,21,...,0.655109,0.649635,0.697874,0.644321,0.657599,0.555556,1,1.0,"[1, 2, 3, 4]",4
2266583,7441,A040165002,A040000165,1,2020-08-21 01:06:50,8836,2,4,165,21,...,0.655109,0.635036,0.697874,0.644321,0.657599,0.555556,1,1.0,"[1, 2, 3, 4]",4
2266584,7441,A040165003,A040000165,1,2020-08-21 01:07:36,8836,3,4,165,21,...,0.655109,0.791971,0.697874,0.644321,0.657599,0.555556,1,1.0,"[1, 2, 3, 4]",4


In [112]:
new_train3.rename(columns = {'question_number_x':'question_number','question_number_y':'question_nums', 'len':'question_numslen'},inplace=True)

In [113]:
new_train3['exp_test'] = new_train3['exp_test'].astype(int)

In [114]:
new_train3

,userID,assessmentItemID,testId,answerCode,Timestamp,KnowledgeTag,question_number,test_cat,test_id,test_day,...,test_acc,question_acc,tag_acc,time_acc,month_acc,user_acc,exp_question,exp_test,question_nums,question_numslen
0,0,A060001001,A060000001,1,2020-03-24 00:17:11,7224,1,6,001,24,...,0.947683,0.982063,0.955022,0.649446,0.682643,0.630872,1,1,"[1, 2, 3, 4, 5, 7]",6
1,0,A060001002,A060000001,1,2020-03-24 00:17:14,7225,2,6,001,24,...,0.947683,0.964126,0.913187,0.649446,0.682643,0.630872,1,1,"[1, 2, 3, 4, 5, 7]",6
2,0,A060001003,A060000001,1,2020-03-24 00:17:22,7225,3,6,001,24,...,0.947683,0.910314,0.913187,0.649446,0.682643,0.630872,1,1,"[1, 2, 3, 4, 5, 7]",6
3,0,A060001004,A060000001,1,2020-03-24 00:17:29,7225,4,6,001,24,...,0.947683,0.968610,0.913187,0.649446,0.682643,0.630872,1,1,"[1, 2, 3, 4, 5, 7]",6
4,0,A060001005,A060000001,1,2020-03-24 00:17:36,7225,5,6,001,24,...,0.947683,0.941704,0.913187,0.649446,0.682643,0.630872,1,1,"[1, 2, 3, 4, 5, 7]",6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2266581,7441,A030071005,A030000071,0,2020-06-05 06:50:21,438,5,3,071,05,...,0.662590,0.435252,0.689706,0.629941,0.651967,0.555556,1,1,"[1, 2, 3, 4, 5]",5
2266582,7441,A040165001,A040000165,1,2020-08-21 01:06:39,8836,1,4,165,21,...,0.655109,0.649635,0.697874,0.644321,0.657599,0.555556,1,1,"[1, 2, 3, 4]",4
2266583,7441,A040165002,A040000165,1,2020-08-21 01:06:50,8836,2,4,165,21,...,0.655109,0.635036,0.697874,0.644321,0.657599,0.555556,1,1,"[1, 2, 3, 4]",4
2266584,7441,A040165003,A040000165,1,2020-08-21 01:07:36,8836,3,4,165,21,...,0.655109,0.791971,0.697874,0.644321,0.657599,0.555556,1,1,"[1, 2, 3, 4]",4


In [115]:
# 태그를 몇 번 푸는지?
exp_tag = new_train3.groupby(['KnowledgeTag'])['userID'].value_counts().reset_index(name='exp_tag')
new_train3 = pd.merge(left = new_train3, right = exp_tag, on = ['userID', 'KnowledgeTag'], how = 'left')
new_train3

,userID,assessmentItemID,testId,answerCode,Timestamp,KnowledgeTag,question_number,test_cat,test_id,test_day,...,question_acc,tag_acc,time_acc,month_acc,user_acc,exp_question,exp_test,question_nums,question_numslen,exp_tag
0,0,A060001001,A060000001,1,2020-03-24 00:17:11,7224,1,6,001,24,...,0.982063,0.955022,0.649446,0.682643,0.630872,1,1,"[1, 2, 3, 4, 5, 7]",6,1
1,0,A060001002,A060000001,1,2020-03-24 00:17:14,7225,2,6,001,24,...,0.964126,0.913187,0.649446,0.682643,0.630872,1,1,"[1, 2, 3, 4, 5, 7]",6,5
2,0,A060001003,A060000001,1,2020-03-24 00:17:22,7225,3,6,001,24,...,0.910314,0.913187,0.649446,0.682643,0.630872,1,1,"[1, 2, 3, 4, 5, 7]",6,5
3,0,A060001004,A060000001,1,2020-03-24 00:17:29,7225,4,6,001,24,...,0.968610,0.913187,0.649446,0.682643,0.630872,1,1,"[1, 2, 3, 4, 5, 7]",6,5
4,0,A060001005,A060000001,1,2020-03-24 00:17:36,7225,5,6,001,24,...,0.941704,0.913187,0.649446,0.682643,0.630872,1,1,"[1, 2, 3, 4, 5, 7]",6,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2266581,7441,A030071005,A030000071,0,2020-06-05 06:50:21,438,5,3,071,05,...,0.435252,0.689706,0.629941,0.651967,0.555556,1,1,"[1, 2, 3, 4, 5]",5,5
2266582,7441,A040165001,A040000165,1,2020-08-21 01:06:39,8836,1,4,165,21,...,0.649635,0.697874,0.644321,0.657599,0.555556,1,1,"[1, 2, 3, 4]",4,4
2266583,7441,A040165002,A040000165,1,2020-08-21 01:06:50,8836,2,4,165,21,...,0.635036,0.697874,0.644321,0.657599,0.555556,1,1,"[1, 2, 3, 4]",4,4
2266584,7441,A040165003,A040000165,1,2020-08-21 01:07:36,8836,3,4,165,21,...,0.791971,0.697874,0.644321,0.657599,0.555556,1,1,"[1, 2, 3, 4]",4,4


In [116]:
new_train3.rename(columns = {'time_acc':'hour_acc'},inplace=True) # 실수 복구

In [117]:
new_train3.columns

Index(['userID', 'assessmentItemID', 'testId', 'answerCode', 'Timestamp',
       'KnowledgeTag', 'question_number', 'test_cat', 'test_id', 'test_day',
       'test_hour', 'test_month', 'test_acc', 'question_acc', 'tag_acc',
       'hour_acc', 'month_acc', 'user_acc', 'exp_question', 'exp_test',
       'question_nums', 'question_numslen', 'exp_tag'],
      dtype='object')

In [118]:
new_train3 = new_train3[['userID', 'assessmentItemID', 'testId', 'answerCode', 'Timestamp', 'KnowledgeTag', 'test_cat',
       'test_id', 'question_number', 'question_numslen', 'test_month', 'test_day', 'test_hour',
       'user_acc', 'test_acc', 'tag_acc', 'question_acc', 'month_acc',
       'hour_acc', 'exp_test', 'exp_tag', 'exp_question']]

In [119]:
new_train3

,userID,assessmentItemID,testId,answerCode,Timestamp,KnowledgeTag,test_cat,test_id,question_number,question_numslen,...,test_hour,user_acc,test_acc,tag_acc,question_acc,month_acc,hour_acc,exp_test,exp_tag,exp_question
0,0,A060001001,A060000001,1,2020-03-24 00:17:11,7224,6,001,1,6,...,00,0.630872,0.947683,0.955022,0.982063,0.682643,0.649446,1,1,1
1,0,A060001002,A060000001,1,2020-03-24 00:17:14,7225,6,001,2,6,...,00,0.630872,0.947683,0.913187,0.964126,0.682643,0.649446,1,5,1
2,0,A060001003,A060000001,1,2020-03-24 00:17:22,7225,6,001,3,6,...,00,0.630872,0.947683,0.913187,0.910314,0.682643,0.649446,1,5,1
3,0,A060001004,A060000001,1,2020-03-24 00:17:29,7225,6,001,4,6,...,00,0.630872,0.947683,0.913187,0.968610,0.682643,0.649446,1,5,1
4,0,A060001005,A060000001,1,2020-03-24 00:17:36,7225,6,001,5,6,...,00,0.630872,0.947683,0.913187,0.941704,0.682643,0.649446,1,5,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2266581,7441,A030071005,A030000071,0,2020-06-05 06:50:21,438,3,071,5,5,...,06,0.555556,0.662590,0.689706,0.435252,0.651967,0.629941,1,5,1
2266582,7441,A040165001,A040000165,1,2020-08-21 01:06:39,8836,4,165,1,4,...,01,0.555556,0.655109,0.697874,0.649635,0.657599,0.644321,1,4,1
2266583,7441,A040165002,A040000165,1,2020-08-21 01:06:50,8836,4,165,2,4,...,01,0.555556,0.655109,0.697874,0.635036,0.657599,0.644321,1,4,1
2266584,7441,A040165003,A040000165,1,2020-08-21 01:07:36,8836,4,165,3,4,...,01,0.555556,0.655109,0.697874,0.791971,0.657599,0.644321,1,4,1


In [120]:
new_train3.columns

Index(['userID', 'assessmentItemID', 'testId', 'answerCode', 'Timestamp',
       'KnowledgeTag', 'test_cat', 'test_id', 'question_number',
       'question_numslen', 'test_month', 'test_day', 'test_hour', 'user_acc',
       'test_acc', 'tag_acc', 'question_acc', 'month_acc', 'hour_acc',
       'exp_test', 'exp_tag', 'exp_question'],
      dtype='object')

In [124]:
time_train = new_train3.copy()
time_train

,userID,assessmentItemID,testId,answerCode,Timestamp,KnowledgeTag,test_cat,test_id,question_number,question_numslen,...,test_hour,user_acc,test_acc,tag_acc,question_acc,month_acc,hour_acc,exp_test,exp_tag,exp_question
0,0,A060001001,A060000001,1,2020-03-24 00:17:11,7224,6,001,1,6,...,00,0.630872,0.947683,0.955022,0.982063,0.682643,0.649446,1,1,1
1,0,A060001002,A060000001,1,2020-03-24 00:17:14,7225,6,001,2,6,...,00,0.630872,0.947683,0.913187,0.964126,0.682643,0.649446,1,5,1
2,0,A060001003,A060000001,1,2020-03-24 00:17:22,7225,6,001,3,6,...,00,0.630872,0.947683,0.913187,0.910314,0.682643,0.649446,1,5,1
3,0,A060001004,A060000001,1,2020-03-24 00:17:29,7225,6,001,4,6,...,00,0.630872,0.947683,0.913187,0.968610,0.682643,0.649446,1,5,1
4,0,A060001005,A060000001,1,2020-03-24 00:17:36,7225,6,001,5,6,...,00,0.630872,0.947683,0.913187,0.941704,0.682643,0.649446,1,5,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2266581,7441,A030071005,A030000071,0,2020-06-05 06:50:21,438,3,071,5,5,...,06,0.555556,0.662590,0.689706,0.435252,0.651967,0.629941,1,5,1
2266582,7441,A040165001,A040000165,1,2020-08-21 01:06:39,8836,4,165,1,4,...,01,0.555556,0.655109,0.697874,0.649635,0.657599,0.644321,1,4,1
2266583,7441,A040165002,A040000165,1,2020-08-21 01:06:50,8836,4,165,2,4,...,01,0.555556,0.655109,0.697874,0.635036,0.657599,0.644321,1,4,1
2266584,7441,A040165003,A040000165,1,2020-08-21 01:07:36,8836,4,165,3,4,...,01,0.555556,0.655109,0.697874,0.791971,0.657599,0.644321,1,4,1


In [125]:
# time_train에서 다른 테스트의 question_number가 같은 문제를 동시에 푼 경우 존재 확인
check = time_train.groupby(['userID', 'Timestamp', 'question_number'])['answerCode'].count().to_frame()
check[check['answerCode'] > 1]

answerCode
userID Timestamp           question_number            
23     2020-08-18 23:40:18 3                         2
467    2020-05-29 05:56:46 2                         2
635    2020-07-08 12:17:43 1                         2
893    2020-08-24 12:42:15 1                         2
1786   2020-10-15 11:20:39 4                         2
1853   2020-10-21 10:45:27 2                         2
2190   2020-08-06 00:21:11 5                         2
4000   2020-08-12 07:21:32 1                         2
       2020-08-12 07:21:34 2                         2
5125   2020-10-19 08:36:16 3                         2

In [126]:
# assessmentItemID로 확인하면 겹치는 것은 없음 -> 완전히 같은 문제를 동시에 푸는 경우는 없다는 말
check2 = time_train.groupby(['userID', 'Timestamp', 'assessmentItemID'])['answerCode'].count().to_frame()
check2[check2['answerCode'] > 1]

,,,answerCode
userID,Timestamp,assessmentItemID,


In [52]:
train[((train['userID'] == 0) & (train['Timestamp'] == '2020-05-07 00:52:47'))]

,userID,assessmentItemID,testId,answerCode,Timestamp,KnowledgeTag
122,0,A080016007,A080000016,1,2020-05-07 00:52:47,4672
123,0,A080016008,A080000016,0,2020-05-07 00:52:47,4672


In [127]:
# 다시 원래의 순서대로 돌리기 위해 index reset은 하지 않음
time_train = time_train.sort_values(by=['userID', 'Timestamp', 'assessmentItemID'])

In [128]:
by_user = time_train.groupby(['userID', 'Timestamp', 'assessmentItemID'])['answerCode'].sum().to_frame()
by_user 

answerCode
userID Timestamp           assessmentItemID            
0      2020-03-24 00:17:11 A060001001                 1
       2020-03-24 00:17:14 A060001002                 1
       2020-03-24 00:17:22 A060001003                 1
       2020-03-24 00:17:29 A060001004                 1
       2020-03-24 00:17:36 A060001005                 1
...                                                 ...
7441   2020-06-05 06:50:21 A030071005                 0
       2020-08-21 01:06:39 A040165001                 1
       2020-08-21 01:06:50 A040165002                 1
       2020-08-21 01:07:36 A040165003                 1
       2020-08-21 01:08:49 A040165004                 1

[2266586 rows x 1 columns]

In [129]:
by_user = by_user.groupby(level=0).cumsum()
by_user

answerCode
userID Timestamp           assessmentItemID            
0      2020-03-24 00:17:11 A060001001                 1
       2020-03-24 00:17:14 A060001002                 2
       2020-03-24 00:17:22 A060001003                 3
       2020-03-24 00:17:29 A060001004                 4
       2020-03-24 00:17:36 A060001005                 5
...                                                 ...
7441   2020-06-05 06:50:21 A030071005                 1
       2020-08-21 01:06:39 A040165001                 2
       2020-08-21 01:06:50 A040165002                 3
       2020-08-21 01:07:36 A040165003                 4
       2020-08-21 01:08:49 A040165004                 5

[2266586 rows x 1 columns]

In [130]:
by_user = by_user.reset_index(drop=True)
by_user['answerCode']

0          1
1          2
2          3
3          4
4          5
          ..
2266581    1
2266582    2
2266583    3
2266584    4
2266585    5
Name: answerCode, Length: 2266586, dtype: int64

In [132]:
time_train['ans_cumsum'] = by_user['answerCode']
time_train = time_train.sort_index() # 원래의 순서대로 돌리기
time_train

,userID,assessmentItemID,testId,answerCode,Timestamp,KnowledgeTag,test_cat,test_id,question_number,question_numslen,...,user_acc,test_acc,tag_acc,question_acc,month_acc,hour_acc,exp_test,exp_tag,exp_question,ans_cumsum
0,0,A060001001,A060000001,1,2020-03-24 00:17:11,7224,6,001,1,6,...,0.630872,0.947683,0.955022,0.982063,0.682643,0.649446,1,1,1,1
1,0,A060001002,A060000001,1,2020-03-24 00:17:14,7225,6,001,2,6,...,0.630872,0.947683,0.913187,0.964126,0.682643,0.649446,1,5,1,2
2,0,A060001003,A060000001,1,2020-03-24 00:17:22,7225,6,001,3,6,...,0.630872,0.947683,0.913187,0.910314,0.682643,0.649446,1,5,1,3
3,0,A060001004,A060000001,1,2020-03-24 00:17:29,7225,6,001,4,6,...,0.630872,0.947683,0.913187,0.968610,0.682643,0.649446,1,5,1,4
4,0,A060001005,A060000001,1,2020-03-24 00:17:36,7225,6,001,5,6,...,0.630872,0.947683,0.913187,0.941704,0.682643,0.649446,1,5,1,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2266581,7441,A030071005,A030000071,0,2020-06-05 06:50:21,438,3,071,5,5,...,0.555556,0.662590,0.689706,0.435252,0.651967,0.629941,1,5,1,1
2266582,7441,A040165001,A040000165,1,2020-08-21 01:06:39,8836,4,165,1,4,...,0.555556,0.655109,0.697874,0.649635,0.657599,0.644321,1,4,1,2
2266583,7441,A040165002,A040000165,1,2020-08-21 01:06:50,8836,4,165,2,4,...,0.555556,0.655109,0.697874,0.635036,0.657599,0.644321,1,4,1,3
2266584,7441,A040165003,A040000165,1,2020-08-21 01:07:36,8836,4,165,3,4,...,0.555556,0.655109,0.697874,0.791971,0.657599,0.644321,1,4,1,4


# 정리
## train
- 원본
## new_train
- 다른 기법 없이 기존 columns만 정리하여 새로운 column 생성
- userID	
- answerCode	
- Timestamp	
- KnowledgeTag	
- question_number	
- test_cat	
- test_id	
- test_day	
- test_hour	
- test_month
## new_train2
- 새로운 기법(완료된 project 참조) 적용해 column 생성
- userID	
- answerCode	
- Timestamp	
- KnowledgeTag	
- question_number	
- test_cat	
- test_id	
- test_day	
- test_hour	
- test_month	
- test_acc	
- question_acc	
- tag_acc	
- time_acc	
- month_acc	
- user_acc
## new_train3 (11/21 월요일)
- 태그, 문제, 테스트 중복 횟수 column 추가~
- userID: 유저 아이디
- answerCode: 정답 여부 1: 정답 0: 오답
- Timestamp: 문제를 풀기 시작한 시간
- KnowledgeTag: 문제 유형 태그
- test_cat: 시험의 대분류
- test_id: 시험의 고유 아이디
- question_number: 시험에서의 문제 번호
   - test_cat, test_id, question_number는 기존의 assessmentItemID를 분해한 것
- question_numslen: 시험에서 몇 개의 번호가 존재하는가 (1,2,3,5가 출제되었다면 question_numslen는 4)
- test_month: 시험을 본 달
- test_day: 시험을 본 일
- test_hour: 시험을 본 시간
   - test_month, test_day, test_hour는 Timestamp에서 의미가 있다고 판단되는 feature
   - 2019/12/31이 일부 있었으나, 2020/01/01로 바꾸어서 test_year가 의미 없어짐
- user_acc: 해당 유저의 전체 정답률
- test_acc: 해당 test의 정답률
- tag_acc: 해당 tag의 정답률
- question_acc: 해당 문제의 정답률
- month_acc: 이 달의 전체 유저의 정답률
- hour_acc: 이 시간의 전체 유저의 정답률
- exp_test: 해당 테스트를 경험한 수 (최소 1 최대 3)
- exp_tag: 해당 태그를 경험한 수
- exp_question: 해당 문제를 경험한 수(대체로 exp_test를 따라갈 것)
## time_train (11/22 화요일)
- ans_cumsum: 시간 순서에 따른 유저의 answer code 누적합